<a href="https://colab.research.google.com/github/amanteur/TDA_Cover_detection/blob/main/Practics/TDA_CoverDetection_Practics_1_Ranking_by_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.2.0
!pip install gudhi

     |████████████████████████████████| 9.9 MB 5.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.2.0 which is incompatible.


     |████████████████████████████████| 28.2 MB 19 kB/s 


In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import gudhi as gd
import gudhi.representations as gdr

In [ ]:
pd.__version__

'1.2.0'

#Extracting data

In [ ]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/CourseWork/pers_feat_datasets/'

In [ ]:
df_dict = {}
for _, _, files in os.walk(root_dir):
  for file in files:
    name = file.split('.')[0]
    df = pd.read_json(root_dir + file)
    df_dict [name] = df

In [ ]:
df_dict['af_orig']

,name,data_persistence,Landscape,Silhouette,Entropy_vector,Betti_curves,Entropy_scalar
0,Blue_Collar_Man,"[[0.1094097146, 0.12137259880000001], [0.12257...","[[0.0027359190000000003, 0.005471838000000001,...","[[0.0, 4.92894e-05, 9.85787e-05, 0.0001478681,...","[[0.1740598488, 0.1740598488, 0.1740598488, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[5.4096585283]
1,Blue_Collar_Man,"[[0.10456191470000001, 0.1074333735], [0.11090...","[[0.0027437115, 0.0054874231, 0.0082311346, 0....","[[0.0, 4.82775e-05, 9.65549e-05, 0.0001448324,...","[[0.18006625310000002, 0.18006625310000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0,...",[5.55205177]
2,Addicted_To_Love,"[[0.0679046642, 0.06818033920000001], [0.07322...","[[0.0027692639, 0.0055385278, 0.0083077918, 0....","[[0.0, 6.9536e-05, 0.0001390719, 0.0002086079,...","[[0.3346180579, 0.3346180579, 0.3346180579, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[8.106067468100001]
3,Addicted_To_Love,"[[0.1058075205, 0.1094096744], [0.1048539877, ...","[[0.0027473118, 0.0054946235, 0.00824193530000...","[[0.0, 8.81113e-05, 0.0001762226, 0.0002835577...","[[0.34514422380000004, 0.34514422380000004, 0....","[[1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",[7.9458391145]
4,All_Tomorrow_s_Parties,"[[0.1020223645, 0.10319306360000001], [0.10279...","[[0.0027412049, 0.0054824097, 0.0082236146, 0....","[[0.0, 5.2211000000000005e-05, 0.0001044219, 0...","[[0.18016593420000002, 0.18016593420000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0,...",[5.4899468328]
...,...,...,...,...,...,...,...
159,What_s_Going_On,"[[0.1759236747, 0.1806943227], [0.1733503382, ...","[[0.0026885189000000003, 0.005377037800000001,...","[[0.0, 7.95728e-05, 0.00015914570000000002, 0....","[[0.309578204, 0.309578204, 0.309578204, 0.615...","[[1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",[7.6816156799]
160,We_Can_Work_It_Out,"[[0.1273760457, 0.133167699], [0.1311880593, 0...","[[0.0026085343, 0.0052170686, 0.0078256029, 0....","[[0.0, 6.34127e-05, 0.0001268254, 0.0001902380...","[[0.36775235110000004, 0.36775235110000004, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[7.0495690237]
161,We_Can_Work_It_Out,"[[0.0834015483, 0.0889094239], [0.0906008435, ...","[[0.0021917821000000002, 0.0043835642000000005...","[[0.0, 0.0001340262, 0.0002680524, 0.000402078...","[[0.31727788560000003, 0.31727788560000003, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[8.1308029254]
162,Yesterday,"[[0.1891314084, 0.2037429114], [0.1981541631, ...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 0.00013198110000000002, 0.0002639622000...","[[0.3191602489, 0.3191602489, 0.3191602489, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[7.9458674476]


#Making batches for one dataset

In [ ]:
df_query = df_dict['af_orig'].drop_duplicates(subset=['name'])
df_search = df_dict['af_orig'][~df_dict['af_orig'].index.isin(df_query.index)].drop_duplicates(subset=['name'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

def data_preprocessing(data):
  data_scaled = gdr.DiagramSelector(use=True, point_type="finite").fit_transform(data)
  data_scaled = gdr.DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())]).fit_transform(data_scaled)
  data_scaled = gdr.ProminentPoints(use=True, num_pts=25).fit_transform(data_scaled)
  return data_scaled

In [ ]:
def norm(data1, data2):
  return np.linalg.norm(np.array(data1)-np.array(data2)) 

In [ ]:
metrics_list = ['Landscape_norm', 
                'Silhouette_norm', 
                'Entropy_norm1', 
                'Betti_curves_norm', 
                'PWG_kernel', 
                'PSS_kernel', 
                'PF_kernel', 
                'SW_kernel', 
                'Bottleneck_distance',
                'Wasserstein_distance']

In [ ]:
pers_col = next(col for col in df_query.columns if 'persistence' in col)
metrics_dict = {}
for index_q, row_q in tqdm(df_query.iterrows(), total=df_query.shape[0], desc = 'Progress'):
  search_index = df_search.index[df_query['name'] == row_q['name']][0]
  feature_dict = {}
  diag_query = np.array(row_q[pers_col])
  diag_X = data_preprocessing([diag_query])
  for index_s, row_s in df_search.iterrows():
    diag_search = np.array(row_s[pers_col])
    diag_Y = data_preprocessing([diag_search])

    #computing norms
    #landscape
    ls_norm = norm(row_q['Landscape'], row_s['Landscape'])

    #silhouette norm 2
    sh_norm = norm(row_q['Silhouette'], row_s['Silhouette'])

    #entropy
    et_norm = norm(row_q['Entropy_vector'], row_s['Entropy_vector'])
    #Betti curves
    bc_norm = norm(row_q['Betti_curves'], row_s['Betti_curves'])

    #distances
    #bottleneck
    BD = gdr.BottleneckDistance(epsilon=.001)
    BD.fit(diag_X)
    bd = BD.transform(diag_Y)

    #wasserstein
    WD = gdr.WassersteinDistance(internal_p=2, order=2)
    WD.fit(diag_X)
    wd = WD.transform(diag_Y)

    #kernels
    ##PWG 
    PWG = gdr.PersistenceWeightedGaussianKernel(bandwidth=0.01, kernel_approx=None,\
                                        weight=lambda x: np.arctan(np.power(x[1], 1)))
    PWG.fit(diag_X)
    pwg = PWG.transform(diag_Y)

    ##PSS
    PSS = gdr.PersistenceScaleSpaceKernel(bandwidth=1.)
    PSS.fit(diag_X)
    pss = PSS.transform(diag_Y)

    ##PF
    PF = gdr.PersistenceFisherKernel(bandwidth_fisher=1., bandwidth=1.)
    PF.fit(diag_X)
    pf = PF.transform(diag_Y)

    ##SW
    SW = gdr.SlicedWassersteinKernel(bandwidth=.1, num_directions=100)
    SW.fit(diag_X)
    sw = SW.transform(diag_Y)

    #adding to dict
    feature_dict.setdefault(index_s, {})
    feature_dict[index_s]['Landscape_norm'] = ls_norm

    feature_dict[index_s]['Silhouette_norm'] = sh_norm

    feature_dict[index_s]['Entropy_norm1'] = et_norm

    feature_dict[index_s]['Betti_curves_norm'] = bc_norm

    feature_dict[index_s]['PWG_kernel'] = pwg[0][0]
    feature_dict[index_s]['PSS_kernel'] = pss[0][0]
    feature_dict[index_s]['PF_kernel'] = pf[0][0]
    feature_dict[index_s]['SW_kernel'] = sw[0][0]

    feature_dict[index_s]['Bottleneck_distance'] = bd[0][0]
    feature_dict[index_s]['Wasserstein_distance'] = wd[0][0]
  dict_df = pd.DataFrame.from_dict(feature_dict).transpose()
  dict_df.index.name = 'search_idx'
  dict_df = dict_df.reset_index()
  #get ranks
  for metric in metrics_list:
    if 'kernel' in metric:
      asc = False
    else:
      asc = True
    sorted_dict_df = dict_df.sort_values(by=metric, ascending=asc)
    sorted_dict_df = sorted_dict_df.reset_index(drop=True)[['search_idx', metric]]
    metrics_dict.setdefault(index_q, {})
    metrics_dict[index_q][metric] = sorted_dict_df.index[sorted_dict_df['search_idx'] == search_index][0] + 1

Progress: 100%|██████████| 80/80 [00:52<00:00,  1.51it/s]


In [ ]:
rank_df =  pd.DataFrame.from_dict(metrics_dict).transpose()

##Mean Average Rank (MAR)

In [ ]:
rank_df.mean()

Landscape_norm          38.6375
Silhouette_norm         38.9125
Entropy_norm1           37.9000
Betti_curves_norm       36.7000
PWG_kernel              36.4000
PSS_kernel              40.5500
PF_kernel               36.8375
SW_kernel               36.9000
Bottleneck_distance     36.3625
Wasserstein_distance    36.7125
dtype: float64

## mean Average Precision

In [ ]:
rank_df_AP = 1 / rank_df
rank_df_AP.mean()

Landscape_norm          0.064422
Silhouette_norm         0.059946
Entropy_norm1           0.075455
Betti_curves_norm       0.077974
PWG_kernel              0.077673
PSS_kernel              0.062137
PF_kernel               0.089186
SW_kernel               0.084813
Bottleneck_distance     0.105970
Wasserstein_distance    0.098133
dtype: float64

# All datasets

In [ ]:
for name, df in df_dict.items():
  if 'DTM' in name:
    continue
  df_query = df.drop_duplicates(subset=['name'])
  df_search = df[~df.index.isin(df_query.index)].drop_duplicates(subset=['name'])

  pers_col = next(col for col in df_query.columns if 'persistence' in col)
  metrics_dict = {}
  for index_q, row_q in tqdm(df_query.iterrows(), total=df_query.shape[0], desc = 'Progress'):
    search_index = df_search.index[df_query['name'] == row_q['name']][0]
    feature_dict = {}
    diag_query = np.array(row_q[pers_col])
    diag_X = data_preprocessing([diag_query])
    for index_s, row_s in df_search.iterrows():
      diag_search = np.array(row_s[pers_col])
      diag_Y = data_preprocessing([diag_search])

      #computing norms
      #landscape
      ls_norm = norm(row_q['Landscape'], row_s['Landscape'])

      #silhouette norm 2
      sh_norm = norm(row_q['Silhouette'], row_s['Silhouette'])

      #entropy
      et_norm = norm(row_q['Entropy_vector'], row_s['Entropy_vector'])
      #Betti curves
      bc_norm = norm(row_q['Betti_curves'], row_s['Betti_curves'])

      #distances
      #bottleneck
      BD = gdr.BottleneckDistance(epsilon=.001)
      BD.fit(diag_X)
      bd = BD.transform(diag_Y)

      #wasserstein
      WD = gdr.WassersteinDistance(internal_p=2, order=2)
      WD.fit(diag_X)
      wd = WD.transform(diag_Y)

      #kernels
      ##PWG 
      PWG = gdr.PersistenceWeightedGaussianKernel(bandwidth=0.01, kernel_approx=None,\
                                          weight=lambda x: np.arctan(np.power(x[1], 1)))
      PWG.fit(diag_X)
      pwg = PWG.transform(diag_Y)

      ##PSS
      PSS = gdr.PersistenceScaleSpaceKernel(bandwidth=1.)
      PSS.fit(diag_X)
      pss = PSS.transform(diag_Y)

      ##PF
      PF = gdr.PersistenceFisherKernel(bandwidth_fisher=1., bandwidth=1.)
      PF.fit(diag_X)
      pf = PF.transform(diag_Y)

      ##SW
      SW = gdr.SlicedWassersteinKernel(bandwidth=.1, num_directions=100)
      SW.fit(diag_X)
      sw = SW.transform(diag_Y)

      #adding to dict
      feature_dict.setdefault(index_s, {})
      feature_dict[index_s]['Landscape_norm'] = ls_norm

      feature_dict[index_s]['Silhouette_norm'] = sh_norm

      feature_dict[index_s]['Entropy_norm1'] = et_norm

      feature_dict[index_s]['Betti_curves_norm'] = bc_norm

      feature_dict[index_s]['PWG_kernel'] = pwg[0][0]
      feature_dict[index_s]['PSS_kernel'] = pss[0][0]
      feature_dict[index_s]['PF_kernel'] = pf[0][0]
      feature_dict[index_s]['SW_kernel'] = sw[0][0]

      feature_dict[index_s]['Bottleneck_distance'] = bd[0][0]
      feature_dict[index_s]['Wasserstein_distance'] = wd[0][0]
    dict_df = pd.DataFrame.from_dict(feature_dict).transpose()
    dict_df.index.name = 'search_idx'
    dict_df = dict_df.reset_index()
    #get ranks
    for metric in metrics_list:
      if 'kernel' in metric:
        asc = False
      else:
        asc = True
      sorted_dict_df = dict_df.sort_values(by=metric, ascending=asc)
      sorted_dict_df = sorted_dict_df.reset_index(drop=True)[['search_idx', metric]]
      metrics_dict.setdefault(index_q, {})
      metrics_dict[index_q][metric] = sorted_dict_df.index[sorted_dict_df['search_idx'] == search_index][0] + 1
  rank_df =  pd.DataFrame.from_dict(metrics_dict).transpose()
  rank_df_AP = 1 / rank_df
  print(name, '\n')
  print('Mean Average Rank')
  print(rank_df.mean())
  print('Mean Average Precision')
  print(rank_df_AP.mean())

Progress: 100%|██████████| 80/80 [00:53<00:00,  1.50it/s]


af_orig 

Mean Average Rank
Landscape_norm          38.6375
Silhouette_norm         38.9125
Entropy_norm1           37.9000
Betti_curves_norm       36.7000
PWG_kernel              36.4000
PSS_kernel              40.5500
PF_kernel               36.8375
SW_kernel               36.9000
Bottleneck_distance     36.3625
Wasserstein_distance    36.7125
dtype: float64
Mean Average Precision
Landscape_norm          0.064422
Silhouette_norm         0.059946
Entropy_norm1           0.075455
Betti_curves_norm       0.077974
PWG_kernel              0.077673
PSS_kernel              0.062137
PF_kernel               0.089186
SW_kernel               0.084813
Bottleneck_distance     0.105970
Wasserstein_distance    0.098133
dtype: float64


Progress: 100%|██████████| 80/80 [00:46<00:00,  1.71it/s]


af_sparse 

Mean Average Rank
Landscape_norm          39.0375
Silhouette_norm         38.8000
Entropy_norm1           39.5500
Betti_curves_norm       39.2000
PWG_kernel              40.0375
PSS_kernel              40.4625
PF_kernel               39.7375
SW_kernel               40.1625
Bottleneck_distance     41.1000
Wasserstein_distance    39.7250
dtype: float64
Mean Average Precision
Landscape_norm          0.058692
Silhouette_norm         0.076769
Entropy_norm1           0.053032
Betti_curves_norm       0.069783
PWG_kernel              0.075317
PSS_kernel              0.062027
PF_kernel               0.069732
SW_kernel               0.065683
Bottleneck_distance     0.075916
Wasserstein_distance    0.068163
dtype: float64


Progress: 100%|██████████| 80/80 [00:51<00:00,  1.56it/s]


...

ts_orig 

Mean Average Rank
Landscape_norm          39.8125
Silhouette_norm         37.4750
Entropy_norm1           37.1250
Betti_curves_norm       36.5750
PWG_kernel              40.3750
PSS_kernel              40.5000
PF_kernel               35.8500
SW_kernel               35.2000
Bottleneck_distance     39.5625
Wasserstein_distance    37.8375
dtype: float64
Mean Average Precision
Landscape_norm          0.063840
Silhouette_norm         0.053224
Entropy_norm1           0.074300
Betti_curves_norm       0.067427
PWG_kernel              0.048694
PSS_kernel              0.062967
PF_kernel               0.094493
SW_kernel               0.068744
Bottleneck_distance     0.073461
Wasserstein_distance    0.066948
dtype: float64


Progress: 100%|██████████| 80/80 [00:39<00:00,  2.04it/s]


ts_opt 

Mean Average Rank
Landscape_norm          35.4750
Silhouette_norm         39.1625
Entropy_norm1           38.4750
Betti_curves_norm       38.5625
PWG_kernel              37.2500
PSS_kernel              40.4250
PF_kernel               38.3625
SW_kernel               38.3625
Bottleneck_distance     38.3375
Wasserstein_distance    37.7500
dtype: float64
Mean Average Precision
Landscape_norm          0.089924
Silhouette_norm         0.072596
Entropy_norm1           0.071979
Betti_curves_norm       0.066455
PWG_kernel              0.051331
PSS_kernel              0.062227
PF_kernel               0.067160
SW_kernel               0.063487
Bottleneck_distance     0.065615
Wasserstein_distance    0.056420
dtype: float64


Progress: 100%|██████████| 80/80 [00:38<00:00,  2.08it/s]

ms_orig 

Mean Average Rank
Landscape_norm          41.9875
Silhouette_norm         41.5875
Entropy_norm1           41.3750
Betti_curves_norm       40.0375
PWG_kernel              38.5375
PSS_kernel              40.5250
PF_kernel               40.6250
SW_kernel               40.0625
Bottleneck_distance     41.2250
Wasserstein_distance    39.3000
dtype: float64
Mean Average Precision
Landscape_norm          0.064871
Silhouette_norm         0.052505
Entropy_norm1           0.066211
Betti_curves_norm       0.083605
PWG_kernel              0.054148
PSS_kernel              0.062078
PF_kernel               0.071151
SW_kernel               0.086883
Bottleneck_distance     0.060255
Wasserstein_distance    0.084329
dtype: float64


In [ ]:

df

,name,data_persistence,Landscape,Silhouette,Entropy_vector,Betti_curves,Entropy_scalar
0,Blue_Collar_Man,"[[0.1094097146, 0.12137259880000001], [0.12257...","[[0.0027359190000000003, 0.005471838000000001,...","[[0.0, 4.92894e-05, 9.85787e-05, 0.0001478681,...","[[0.1740598488, 0.1740598488, 0.1740598488, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[5.4096585283]
1,Blue_Collar_Man,"[[0.10456191470000001, 0.1074333735], [0.11090...","[[0.0027437115, 0.0054874231, 0.0082311346, 0....","[[0.0, 4.82775e-05, 9.65549e-05, 0.0001448324,...","[[0.18006625310000002, 0.18006625310000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0,...",[5.55205177]
2,Addicted_To_Love,"[[0.0679046642, 0.06818033920000001], [0.07322...","[[0.0027692639, 0.0055385278, 0.0083077918, 0....","[[0.0, 6.9536e-05, 0.0001390719, 0.0002086079,...","[[0.3346180579, 0.3346180579, 0.3346180579, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[8.106067468100001]
3,Addicted_To_Love,"[[0.1058075205, 0.1094096744], [0.1048539877, ...","[[0.0027473118, 0.0054946235, 0.00824193530000...","[[0.0, 8.81113e-05, 0.0001762226, 0.0002835577...","[[0.34514422380000004, 0.34514422380000004, 0....","[[1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",[7.9458391145]
4,All_Tomorrow_s_Parties,"[[0.1020223645, 0.10319306360000001], [0.10279...","[[0.0027412049, 0.0054824097, 0.0082236146, 0....","[[0.0, 5.2211000000000005e-05, 0.0001044219, 0...","[[0.18016593420000002, 0.18016593420000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0,...",[5.4899468328]
...,...,...,...,...,...,...,...
159,What_s_Going_On,"[[0.1759236747, 0.1806943227], [0.1733503382, ...","[[0.0026885189000000003, 0.005377037800000001,...","[[0.0, 7.95728e-05, 0.00015914570000000002, 0....","[[0.309578204, 0.309578204, 0.309578204, 0.615...","[[1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",[7.6816156799]
160,We_Can_Work_It_Out,"[[0.1273760457, 0.133167699], [0.1311880593, 0...","[[0.0026085343, 0.0052170686, 0.0078256029, 0....","[[0.0, 6.34127e-05, 0.0001268254, 0.0001902380...","[[0.36775235110000004, 0.36775235110000004, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[7.0495690237]
161,We_Can_Work_It_Out,"[[0.0834015483, 0.0889094239], [0.0906008435, ...","[[0.0021917821000000002, 0.0043835642000000005...","[[0.0, 0.0001340262, 0.0002680524, 0.000402078...","[[0.31727788560000003, 0.31727788560000003, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[8.1308029254]
162,Yesterday,"[[0.1891314084, 0.2037429114], [0.1981541631, ...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 0.00013198110000000002, 0.0002639622000...","[[0.3191602489, 0.3191602489, 0.3191602489, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[7.9458674476]
